In [ ]:
from ultralytics import YOLO
import torch
import pandas as pd
import matplotlib.pyplot as plt
import os # Tarvitaan polkujen käsittelyyn
import time # Aikaleimaa varten

# --- Laitteen valinta ---
if torch.cuda.is_available():
    device = 0 # Käytä ensimmäistä GPU:ta
    print(f"Käytetään GPU:ta ({torch.cuda.get_device_name(device)})")
else:
    device = "cpu"
    print(f"Käytetään CPU:ta")

In [ ]:
model = YOLO("yolo11n.pt")
#results = model.train(data='config/materials.yaml', epochs=50, imgsz=640, device=device)


# --- Koulutusparametrien määrittely ---
data_config = 'config/materials.yaml' # POLKU OMAAN DATASETTIMÄÄRITTELYYSI
epochs = 10          # Epochien määrä (säädä tarpeen mukaan)
imgsz = 640           # Koulutuskuvakoko
patience = 5         # Kuinka monta epochia odotetaan ilman parannusta ennen keskeytystä
optimizer = 'AdamW'   # Optimointialgoritmi (oletus usein hyvä)
augment = True        # Käytä datan augmentointia (suositeltavaa)

# === MUUTETUT PARAMETRIT ===
batch_size = 8        # Eräkoko (batch size) asetettu arvoon 8
lr0 = 0.001           # Oppimisnopeuden alkuarvo asetettu 0.001

# Nimeä koulutusajo ja projektikansio 
project_name = 'rummu_detection_training' # Kansion nimi, johon tulokset tallennetaan
run_name = f'run_{time.strftime("%Y%m%d_%H%M%S")}_epochs{epochs}_batch{batch_size}_lr{lr0}' # Uniikki nimi ajolle


# --- Mallin Koulutus ---
# results-objekti sisältää tietoa koulutuksesta
results = model.train(
    data=data_config,
    epochs=epochs,
    imgsz=imgsz,
    device=device,
    batch=batch_size,
    patience=patience,
    optimizer=optimizer,
    lr0=lr0,
    augment=augment,
    project=project_name, # Määrittää pääkansion
    name=run_name         # Määrittää tämän ajon alikansion nimen
)
print(f"\nKoulutus valmis!")
# results.save_dir sisältää polun kansion, johon tämän ajon tulokset tallennettiin
save_directory = results.save_dir
print(f"Tulokset tallennettu kansioon: {save_directory}")


In [ ]:
# --- Validointi parhaalla mallilla (varmistus) ---
print("\nAjetaan validointi parhaalla tallennetulla mallilla...")
best_model_path = os.path.join(save_directory, 'weights/best.pt')
if os.path.exists(best_model_path):
    model_best = YOLO(best_model_path)
    metrics = model_best.val(data=data_config, imgsz=imgsz, device=device) # Aja validointi

    # Tulosta tärkeimmät metriikat
    print("\nValidointimetriikat (parhaasta mallista):")
    if hasattr(metrics, 'box') and hasattr(metrics.box, 'map'):
        print(f"  mAP50-95 (box): {metrics.box.map:.4f}")
        print(f"  mAP50 (box):    {metrics.box.map50:.4f}")
        print(f"  mAP75 (box):    {metrics.box.map75:.4f}")
        if hasattr(metrics.box, 'maps') and metrics.box.maps is not None:
             # Muunnetaan tensorit listoiksi ja pyöristetään selkeyden vuoksi
             maps_list = [round(float(m), 4) for m in metrics.box.maps]
             print(f"  mAP50-95 per luokka: {maps_list}")
    else:
         print("  Validointimetriikoita (box) ei löytynyt.")

else:
    print(f"Virhe: Parasta mallia ei löytynyt polusta: {best_model_path}")


In [ ]:
# --- Tulosten Plottaaminen Manuaalisesti CSV-tiedostosta ---
results_csv_path = os.path.join(save_directory, 'results.csv')

if os.path.exists(results_csv_path):
    print(f"\nPlotataan tuloksia tiedostosta: {results_csv_path}")
    try:
        # Lue CSV-tiedosto DataFrameen
        df = pd.read_csv(results_csv_path)
        # Poista mahdolliset ylimääräiset välilyönnit sarakkeiden nimistä
        df.columns = df.columns.str.strip()

        print("CSV-tiedoston sarakkeet:", df.columns.tolist())

        # Etsi oikeat sarakkeet plottausta varten (nimet voivat vaihdella hieman)
        epoch_col = 'epoch'
        train_box_loss_col = next((col for col in df.columns if col.endswith('train/box_loss')), None)
        val_box_loss_col = next((col for col in df.columns if col.endswith('val/box_loss')), None)
        train_cls_loss_col = next((col for col in df.columns if col.endswith('train/cls_loss')), None)
        val_cls_loss_col = next((col for col in df.columns if col.endswith('val/cls_loss')), None)
        map50_95_col = next((col for col in df.columns if 'metrics/mAP50-95(B)' in col), None)
        map50_col = next((col for col in df.columns if 'metrics/mAP50(B)' in col), None)

        # Varmistus vanhemmille tai hieman eri nimille
        if map50_col is None:
            map50_col = next((col for col in df.columns if 'mAP_0.5' in col and '0.95' not in col), None) # Etsi 'mAP_0.5' jos 'metrics/mAP50(B)' puuttuu
        if map50_95_col is None:
             map50_95_col = next((col for col in df.columns if 'mAP_0.5:0.95' in col), None) # Etsi 'mAP_0.5:0.95' jos 'metrics/mAP50-95(B)' puuttuu

        plt.figure(figsize=(12, 10))

        # --- Plottaa Loss-käyrät ---
        plt.subplot(2, 1, 1)
        plot_loss = False
        if epoch_col in df.columns:
            if train_box_loss_col:
                plt.plot(df[epoch_col], df[train_box_loss_col], label='Training Box Loss')
                plot_loss = True
            if val_box_loss_col:
                plt.plot(df[epoch_col], df[val_box_loss_col], label='Validation Box Loss')
                plot_loss = True
            if train_cls_loss_col:
                plt.plot(df[epoch_col], df[train_cls_loss_col], label='Training Class Loss', linestyle='--')
                plot_loss = True
            if val_cls_loss_col:
                plt.plot(df[epoch_col], df[val_cls_loss_col], label='Validation Class Loss', linestyle='--')
                plot_loss = True

            if plot_loss:
                plt.title('Loss Over Epochs')
                plt.xlabel('Epoch')
                plt.ylabel('Loss')
                plt.legend()
                plt.grid(True)
            else:
                plt.text(0.5, 0.5, 'Loss data not found in results.csv', horizontalalignment='center', verticalalignment='center')
                plt.title('Loss Over Epochs')
        else:
             plt.text(0.5, 0.5, 'Epoch column not found', horizontalalignment='center', verticalalignment='center')
             plt.title('Loss Over Epochs')


        # --- Plottaa mAP-käyrät ---
        plt.subplot(2, 1, 2)
        plot_map = False
        if epoch_col in df.columns:
            if map50_95_col:
                 plt.plot(df[epoch_col], df[map50_95_col], label='Validation mAP50-95')
                 plot_map = True
            if map50_col:
                plt.plot(df[epoch_col], df[map50_col], label='Validation mAP50')
                plot_map = True

            if plot_map:
                plt.title('Mean Average Precision (mAP) Over Epochs')
                plt.xlabel('Epoch')
                plt.ylabel('mAP')
                # Aseta y-akselin raja-arvot järkeviksi (0-1)
                min_y = df[map50_col].min() if map50_col and df[map50_col].notna().any() else 0
                max_y = df[map50_col].max() if map50_col and df[map50_col].notna().any() else 1
                if map50_95_col and df[map50_95_col].notna().any():
                    min_y = min(min_y, df[map50_95_col].min())
                    max_y = max(max_y, df[map50_95_col].max())

                plt.ylim(max(0, min_y - 0.05), min(1, max_y + 0.05)) # Zoomaa hieman, mutta pysy 0-1 välillä
                plt.legend()
                plt.grid(True)
            else:
                plt.text(0.5, 0.5, 'mAP data not found in results.csv', horizontalalignment='center', verticalalignment='center')
                plt.title('Mean Average Precision (mAP) Over Epochs')
        else:
             plt.text(0.5, 0.5, 'Epoch column not found', horizontalalignment='center', verticalalignment='center')
             plt.title('Mean Average Precision (mAP) Over Epochs')


        plt.tight_layout() # Asettele subplotit nätisti
        # Tallenna kuvaaja samaan kansioon kuin muut tulokset
        plot_save_path = os.path.join(save_directory, 'custom_training_plots.png')
        plt.savefig(plot_save_path)
        print(f"Oma kuvaaja tallennettu: {plot_save_path}")
        # Voit myös näyttää kuvan heti poistamalla kommentin seuraavalta riviltä:
        # plt.show()

    except Exception as e:
        print(f"Virhe plotatessa tuloksia CSV:stä: {e}")
        print(f"Tarkista tiedosto: {results_csv_path} ja sen sarakkeiden nimet.")

else:
    print(f"Virhe: Tulostiedostoa 'results.csv' ei löytynyt kansiosta {save_directory}")
    print("Plottausta ei voida suorittaa.")

print("\nSkripti päättyi.")